<a href="https://colab.research.google.com/github/AlexeyTri/PROJECT_SCOOL/blob/master/ActualGraphic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import openpyxl
from openpyxl.cell.cell import WriteOnlyCell
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl import Workbook

import pandas as pd

# Предобработка графика работ подрядчика

In [68]:
from google.colab import auth
auth.authenticate_user()
import re
import gspread
from google.auth import default
creds, _ = default()
import numpy as np
gc = gspread.authorize(creds)
from datetime import datetime
worksheet = gc.open('План работ Договор Адамант')

In [69]:
graf = worksheet.worksheet("ГРАФИК РАБОТ")
graf = graf.get_all_values()
graf = np.array(graf)
graf =  pd.DataFrame(graf[1:, :])
graf

,0,1,2,3,4,5,6,7,8,9,...,282,283,284,285,286,287,288,289,290,291
0,PLAN,ГРАФИК РАБОТ,Начало работ по ГРП,Завершение работ по ГРП,Длительность,Ед.изм.,Объем по контракту,План на 15.12,Факт на 15.12,% выполнения плана,...,23.02.2023,23.02.2023,24.02.2023,24.02.2023,25.02.2023,25.02.2023,26.02.2023,26.02.2023,27.02.2023,27.02.2023
1,***,,,,,,,,,,...,план,факт,план,факт,план,факт,план,факт,план,факт
2,ВVZO_POS1_POD__.,Мобилизация,10.10.2022,10.05.2024,579,--,,,,,...,,,,,,,,,,
3,VZO_POS1_POD_VRD_DP17.01,Временные дороги из дорожных плит (3х1.75),26.10.2022,18.02.2023,116,м2,300,"140,0","140,0","100,00%",...,,,,,,,,,,
4,VZO_POS1_POD_VRS_ES0.01,Временные электросети - установленные на начал...,28.11.2022,12.12.2022,15,комплект,1,"0,0","0,0",,...,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
310,BOT_GP,Выдача рабочего проекта по разделу,18.11.2022,18.11.2022,1,,,"0,0","0,0",,...,,,,,,,,,,
311,_EXP___.,Ввод объекта в эксплуатацию,14.03.2024,06.06.2024,85,,,"0,0","0,0",,...,,,,,,,,,,
312,_ZOS___.,ЗОС,14.03.2024,24.04.2024,42,,,"0,0","0,0",,...,,,,,,,,,,
313,_RNV___.,Ввод объекта в эксплуатацию ( РНВ),25.04.2024,06.06.2024,43,,,"0,0","0,0",,...,,,,,,,,,,


In [74]:
list1 = graf.iloc[:, 1].values
list1

array(['ГРАФИК РАБОТ', '', 'Мобилизация',
       'Временные дороги из дорожных плит (3х1.75)',
       'Временные электросети - установленные на начальном этапе (включают только подключение бытового городка от точки подключения и 1 временных электрощита на территории 1 кран )',
       'Временные электросети - установленные на начальном этапе (1 временный  электрощит на территории  2 ой кран )',
       'Временный водопровод и канализация (с подогревом в зимнее время) - установленные на начальном этапе (включают подключение бытового городка и 2-х санитарных контейнеров)',
       'Устройство основания под бытовые помещения (покрытие из дорожных плит (3х1.75).',
       'Устройство арматурного цеха',
       'Устройство временных скользящих площадок под складирование строительных материалов на перекрытии подземной части (покрытие из дорожных плит (3х1.75)',
       'Изготовление и монтаж информационного стенда',
       'Блок-контейнер - офисное помещение (амортизация). 16 штук',
       'Бытовы

# Предобработка графика Заказчика

In [76]:
from google.colab import files
upload = files.upload()

Saving Сводный план-график 2022 12 11 иса.xlsx to Сводный план-график 2022 12 11 иса (1).xlsx


In [77]:
import pandas as pd

In [78]:
!ls

 openpyxl_stream.xlsx
 sample_data
'Сводный план-график 2022 12 11 иса (1).xlsx'
'Сводный план-график 2022 12 11 иса.xlsx'
'Сводный план-график 2022 12 12 иса.xlsx'


ПАНДАС из редакции графика **заказчика**

In [79]:
df_graph = pd.read_excel(open('Сводный план-график 2022 12 11 иса.xlsx', 'rb'))
df_graph.head()

,Внесено изменение,СДР,Название задачи,%,План %,Откл. План%,Столбец2,Столбец1,Начало,Окончание,Отв. Исполнитель,Названия ресурсов,Unnamed: 12,Unnamed: 13
0,NaN,NaN,Статус задач на 11.12.2022,NaN,NaN,NaN,NaN,NaN,NaN,"1. В фильтре по столбцу ""Названия ресурсов"" на...",NaN,NaN,NaN,1. Уровень. Группа
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2 Уровень. Суммарная задача
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Внесено изменение,СДР,Название задачи,%,План %,Откл. План%,Столбец2,Столбец1,Начало,Окончание,Отв. Исполнитель,Названия ресурсов,NaN,NaN
4,NaN,0,Проект Школа Новый Вгляд 2022 11 28,0.17,23,6,Вт 18.01.22,Ср 10.12.25,2022-01-18 00:00:00,2025-12-10 00:00:00,NaN,NaN,NaN,NaN


In [102]:
list2 = df_graph['Название задачи']
list2 = list2.map(lambda z: str(z).lstrip())
list2 = list2.to_list()
len(list2)

1255

In [124]:
result = list(set(list1) & set(list2))
result[0]

'Монтаж труб-распорок в осях 5-16/Ж-И'

In [126]:
# graf.iloc[:, 1][46] == result[0]
graf[graf.iloc[:, 1] == result[0]]

,0,1,2,3,4,5,6,7,8,9,...,282,283,284,285,286,287,288,289,290,291
46,OOS_KR1_MK0_MRS_TR.03,Монтаж труб-распорок в осях 5-16/Ж-И,25.12.2022,10.01.2023,17,тн,"10,48","0,0","0,0",,...,,,,,,,,,,


In [81]:
# ws2 = wb.active
# for row in df.values.tolist():
#     ws2.append(row)
# wb.save("openpyxl_stream.xlsx")

In [82]:
# df.to_excel('Сводный план-график 2022 12 12 иса.xlsx', sheet_name='Лист1',index=False)